## Práctica de Inteligencia Artificial

La práctica consiste en entrenar un algoritmo para que calcule el precio de un jugador de fútbol del juego FIFA a partir de un dataset del mismo juego.

### Herramientas

Las herramientas que vamos a usar son las librerías Pandas, Numpy y Scikit Learn de Python.

In [1]:
import os

#separamos el dataframe en train y test
from sklearn.model_selection import train_test_split
#vamos a usar un modelo lineal
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pandas as pd
import numpy as np

### Leer y analizar los datos
Ahora podemos leer el dataframe.

In [2]:
df = pd.read_csv(os.path.join("..", "in", "fifa.csv"))
df

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,18202,238813,J. Lundstram,19,https://cdn.sofifa.org/players/4/19/238813.png,England,https://cdn.sofifa.org/flags/14.png,47,65,Crewe Alexandra,...,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K
18203,18203,243165,N. Christoffersson,19,https://cdn.sofifa.org/players/4/19/243165.png,Sweden,https://cdn.sofifa.org/flags/46.png,47,63,Trelleborgs FF,...,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0,€113K
18204,18204,241638,B. Worman,16,https://cdn.sofifa.org/players/4/19/241638.png,England,https://cdn.sofifa.org/flags/14.png,47,67,Cambridge United,...,41.0,32.0,13.0,11.0,6.0,5.0,10.0,6.0,13.0,€165K
18205,18205,246268,D. Walker-Rice,17,https://cdn.sofifa.org/players/4/19/246268.png,England,https://cdn.sofifa.org/flags/14.png,47,66,Tranmere Rovers,...,46.0,20.0,25.0,27.0,14.0,6.0,14.0,8.0,9.0,€143K


Ahora tenemos que buscar las columnas que no tienen relevancia y eliminarlas.

In [3]:
df = df.iloc[:, 2:]
df = df.drop('Photo', 1)
df = df.drop('Flag', 1)
df = df.drop('Club Logo', 1)
df = df.drop('Joined', 1)
df = df.drop('Loaned From', 1)
df = df.drop('Contract Valid Until', 1)
df = df.drop('Real Face', 1)
df = df.drop('Jersey Number', 1)
df = df.drop('Release Clause', 1)
df = df.drop('Name', 1)

Hay algunas columnas en las cuales en vez de un valor hay un NaN. Por ejemplo, algunos jugadores no tienen un club asignado. Podemos borrarlos ya que son pocos. También podemos borrar a los porteros que no tienen asignadas las características LS, ST, etc.

In [4]:
df = df.dropna(subset = ['Club'])
df = df.dropna(subset = ['LS'])

Como vemos, las columnas que por lógica deberían contener un número (Value, Wage), contienen una cadena de caracteres.
Tenemos que pasar estos valores a entero con una función.

In [5]:
def value_to_float(x):
    x = x.replace('€', '')
    ret_val = 0.0
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

In [6]:
df["Value"] = df["Value"].apply(value_to_float)
df["Wage"] = df["Wage"].apply(value_to_float)

Ahora convertimos la columna Weight en un entero.

In [7]:
def weight_to_float(x):
    x = x.replace("lbs", "")
    ret_val = float(x)
    return ret_val

In [8]:
df["Weight"] = df["Weight"].apply(weight_to_float)

Hacemos lo mismo con la columna Height

In [9]:
def height_to_float(x):
    x = x.replace("'", ".")
    ret_val = float(x)
    return ret_val
    

In [10]:
df["Height"] = df["Height"].apply(height_to_float)

Ahora podemos convertir las columnas Nationality, Club, Preferred Foot, Work Rate, Body Type y Position en una estructura hot-encoding.

In [11]:
nat = df.pop("Nationality")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(nat, prefix='nat').reset_index(drop=True)], axis = 1, sort = False)

clb = df.pop("Club")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(clb, prefix='clb').reset_index(drop=True)], axis = 1, sort = False)

preffoot = df.pop("Preferred Foot")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(preffoot, prefix='preffoot').reset_index(drop=True)], axis = 1, sort = False)

wrate = df.pop("Work Rate")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(wrate, prefix='wrate').reset_index(drop=True)], axis = 1, sort = False)

btype = df.pop("Body Type")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(btype, prefix='btype').reset_index(drop=True)], axis = 1, sort = False)

pos = df.pop("Position")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(pos, prefix='pos').reset_index(drop=True)], axis = 1, sort = False)

Ahora podemos convertir las columnas con las características en formato numérico

In [12]:
def car_to_float(x):
    firstSum =  x[:2]
    secondSum =  x[-1]
    ret_val = float(firstSum) + float(secondSum)
    return ret_val


In [13]:
df["LS"] = df["LS"].apply(car_to_float)
df["ST"] = df["ST"].apply(car_to_float)
df["RS"] = df["RS"].apply(car_to_float)
df["LW"] = df["LW"].apply(car_to_float)
df["LF"] = df["LF"].apply(car_to_float)
df["CF"] = df["CF"].apply(car_to_float)
df["RF"] = df["RF"].apply(car_to_float)
df["RW"] = df["RW"].apply(car_to_float)
df["LAM"] = df["LAM"].apply(car_to_float)
df["CAM"] = df["CAM"].apply(car_to_float)
df["RAM"] = df["RAM"].apply(car_to_float)
df["LM"] = df["LM"].apply(car_to_float)
df["LCM"] = df["LCM"].apply(car_to_float)
df["CM"] = df["CM"].apply(car_to_float)
df["RCM"] = df["RCM"].apply(car_to_float)
df["RM"] = df["RM"].apply(car_to_float)
df["LWB"] = df["LWB"].apply(car_to_float)
df["LDM"] = df["LDM"].apply(car_to_float)
df["CDM"] = df["CDM"].apply(car_to_float)
df["RDM"] = df["RDM"].apply(car_to_float)
df["RWB"] = df["RWB"].apply(car_to_float)
df["LB"] = df["LB"].apply(car_to_float)
df["LCB"] = df["LCB"].apply(car_to_float)
df["CB"] = df["CB"].apply(car_to_float)
df["RCB"] = df["RCB"].apply(car_to_float)
df["RB"] = df["RB"].apply(car_to_float)
df


,Age,Overall,Potential,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Height,...,pos_RB,pos_RCB,pos_RCM,pos_RDM,pos_RF,pos_RM,pos_RS,pos_RW,pos_RWB,pos_ST
0,31,94,94,110500000.0,565000.0,2202,5.0,4.0,4.0,5.70,...,0,0,0,0,1,0,0,0,0,0
1,33,94,94,77000000.0,405000.0,2228,5.0,4.0,5.0,6.20,...,0,0,0,0,0,0,0,0,0,1
2,26,92,93,118500000.0,290000.0,2143,5.0,5.0,5.0,5.90,...,0,0,0,0,0,0,0,0,0,0
3,27,91,92,102000000.0,355000.0,2281,4.0,5.0,4.0,5.11,...,0,0,1,0,0,0,0,0,0,0
4,27,91,91,93000000.0,340000.0,2142,4.0,4.0,4.0,5.80,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15921,19,47,65,60000.0,1000.0,1307,1.0,2.0,2.0,5.90,...,0,0,0,0,0,0,0,0,0,0
15922,19,47,63,60000.0,1000.0,1098,1.0,2.0,2.0,6.30,...,0,0,0,0,0,0,0,0,0,1
15923,16,47,67,60000.0,1000.0,1189,1.0,3.0,2.0,5.80,...,0,0,0,0,0,0,0,0,0,1
15924,17,47,66,60000.0,1000.0,1228,1.0,3.0,2.0,5.10,...,0,0,0,0,0,0,0,1,0,0


### Predecir el precio
Ahora tenemos preparado el dataframe y podemos separar la columna que queremos predecir.

In [14]:
val = df.pop("Value")
#df = df.drop("Value", 1)
df

,Age,Overall,Potential,Wage,Special,International Reputation,Weak Foot,Skill Moves,Height,Weight,...,pos_RB,pos_RCB,pos_RCM,pos_RDM,pos_RF,pos_RM,pos_RS,pos_RW,pos_RWB,pos_ST
0,31,94,94,565000.0,2202,5.0,4.0,4.0,5.70,159.0,...,0,0,0,0,1,0,0,0,0,0
1,33,94,94,405000.0,2228,5.0,4.0,5.0,6.20,183.0,...,0,0,0,0,0,0,0,0,0,1
2,26,92,93,290000.0,2143,5.0,5.0,5.0,5.90,150.0,...,0,0,0,0,0,0,0,0,0,0
3,27,91,92,355000.0,2281,4.0,5.0,4.0,5.11,154.0,...,0,0,1,0,0,0,0,0,0,0
4,27,91,91,340000.0,2142,4.0,4.0,4.0,5.80,163.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15921,19,47,65,1000.0,1307,1.0,2.0,2.0,5.90,134.0,...,0,0,0,0,0,0,0,0,0,0
15922,19,47,63,1000.0,1098,1.0,2.0,2.0,6.30,170.0,...,0,0,0,0,0,0,0,0,0,1
15923,16,47,67,1000.0,1189,1.0,3.0,2.0,5.80,148.0,...,0,0,0,0,0,0,0,0,0,1
15924,17,47,66,1000.0,1228,1.0,3.0,2.0,5.10,154.0,...,0,0,0,0,0,0,0,1,0,0


Ya podemos dividir los datos en train/test y ejecutar el aprendizaje automático.

In [15]:
x_train, x_test, y_train, y_test = train_test_split(df, val, test_size = 0.3, random_state = 42)

In [16]:
len(x_train)

11148

In [17]:
reg = linear_model.LinearRegression().fit(x_train, y_train)

In [18]:
preds = reg.predict(x_test)

In [19]:
preds[0]

-107597.13026269898

In [20]:
y_test[0]

110500000.0

In [21]:
r2_score(preds, y_test)

0.799164988714545

Como podemos ver, el entrenamiento ha fallado. Vamos a ver si tenemos algun error en las columnas (NaN)

In [22]:
df.columns[df.isna().any()].tolist()

[]

Como no hay columnas con errores, quedan 2 variantes a ver: puede que el algoritmo de aprendizaje automático se haya equivocado debido a la gran variedad de las columnas y la gran dispersión de los valores o, sinó, podemos deducir que no se puede calcular el precio de un jugador debido a que no hay bastante relación entre el precio y las otras características.